In [1]:
import os

# cache_dir = "/Users/yangye/models"
# 设置环境变量
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 检查环境变量是否设置成功
print(os.environ["HF_ENDPOINT"])
# 设置 CUDA_VISIBLE_DEVICES 环境变量
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

https://hf-mirror.com


In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/yangye/miniconda3/envs/nlp_learn/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
raw_datasets["train"][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [5]:
raw_datasets["train"].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [6]:
from transformers import AutoTokenizer
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return  tokenizer(
        example["sentence1"],example["sentence2"], padding="max_length", truncation=True,max_length=128
    )

tokenizer_datasets=raw_datasets.map(tokenize_function,batched=True)
print(tokenizer_datasets.column_names)

{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [7]:
tokenizer_datasets=tokenizer_datasets.remove_columns(["idx"])
tokenizer_datasets=tokenizer_datasets.rename_column("label","labels")
tokenizer_datasets=tokenizer_datasets.with_format("torch")

tokenizer_datasets["train"]

Dataset({
    features: ['sentence1', 'sentence2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [10]:
# tokenizer_datasets["train"].select(range(1000))

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenizer_datasets["train"], shuffle=True, batch_size=16)

for step,batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step>5:
        break

torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])


In [14]:
from datasets import load_dataset
from  transformers import AutoTokenizer

raw_datasets=load_dataset("glue","mrpc")
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],truncation=True)

tokenizer_datasets=raw_datasets.map(tokenize_function,batched=True)
tokenizer_datasets=tokenizer_datasets.remove_columns(["idx","sentence1","sentence2"])
tokenizer_datasets=tokenizer_datasets.rename_column("label","labels")
tokenizer_datasets=tokenizer_datasets.with_format("torch")

tokenizer_datasets["train"]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [15]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator=DataCollatorWithPadding(tokenizer)
train_dataloader=DataLoader(tokenizer_datasets["train"],shuffle=True,batch_size=8,collate_fn=data_collator)

for step,batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step>5:
        break


torch.Size([8, 71])
torch.Size([8, 69])
torch.Size([8, 72])
torch.Size([8, 73])
torch.Size([8, 72])
torch.Size([8, 71])
torch.Size([8, 84])
